# Route optimization - Openrouteservice 
WasteContainer route optimization with Openrouteservice based on [Vroom](http://vroom-project.org/)

## Authentication

In [4]:
from services.Auth import Auth

# Define service & subservice
service = "tef_vlci"
subservice = "/residuos_contenedores_vlc"

# Authenticate
auth = Auth()
token = auth.get_auth_token_subservice(service, subservice)

/home/joan/citcom-demos/venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'auth.vlci.valencia.es'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Preparing data

In [6]:
from services.BrokerLD import BrokerLD
from shapely.geometry import Point, Polygon

# New Broker NGSI-LD connection
broker = BrokerLD()

# Fetch all WasteContainer entities
limit = 400  # Entites per page (TODO: Implement pagination with response.headers["fiware-total-count"] & offset)
data = broker.get_all_entities_by_type(
    "WasteContainer", service, subservice, token, limit
).json()

# "Campanar" district polygon as sample target area to filter containers
# Set coords
campanar_district_coords = [
    (39.49461650992619, -0.4021854632314645),
    (39.49227149969146, -0.4087410826281664),
    (39.49603047599918, -0.4104111738682555),
    (39.49619157044176, -0.41068952240827034),
    (39.496030475988825, -0.4195270886278404),
    (39.49248630380253, -0.42885176471833775),
    (39.490338232723275, -0.4262770407232004),
    (39.487008591395686, -0.42335438105304446),
    (39.48147673802639, -0.41096787102238386),
    (39.47900605943088, -0.4130554850724952),
    (39.47562216155785, -0.4105503482123616),
    (39.47320499086559, -0.40672305578715745),
    (39.482926878180564, -0.38195003560699226),
    (39.48851232023983, -0.3896046204574005),
    (39.485880773621716, -0.3960762240127457),
]

# Reverse coords to fit geospatial system
i = 0
while i < len(campanar_district_coords):
    campanar_district_coords[i] = reversed(campanar_district_coords[i])
    i += 1

# Create polygon from coords
campanar_district = Polygon(campanar_district_coords)

coords = []

# Filter data
containers = []

for container in data:
    # Discard containers with null location values
    if "location" in container and container["location"]["value"] is not None:
        # Create point from container location
        point = Point(container["location"]["value"]["coordinates"])
        # Check if the container is within target polygon
        if point.within(campanar_district):
            containers.append(container)
            coords.append(container["location"]["value"]["coordinates"])

## Printing entities on map

In [7]:
import folium

# Create map
map = folium.Map(
    location=list([39.47345473371031, -0.3772689881177383]),
    tiles="cartodbpositron",
    zoom_start=14,
)

# For eah container, add a marker with a simple tooltip
for container in containers:
    location = container["location"]["value"]["coordinates"]
    
    filling_level = "-"
    if "fillingLevel" in container:
        filling_level = container["fillingLevel"]["value"]

    folium.Marker(
        location=list(reversed(location)),
        icon=folium.Icon(icon="dumpster", prefix="fa"),
        tooltip="fillingLevel: " + str(filling_level),
    ).add_to(map)

# Print campanar district
folium.GeoJson(
    campanar_district,
    style_function=lambda x: {
        "color": "red",
        "opacity": 0.30,
        "weight": "2",
        "dashArray": "3, 6",
    },
).add_to(map)

map

## Optimization

In [8]:
import os
import openrouteservice as ors

# Openroute service API key
client = ors.Client(key=os.environ.get("OPENROUTESERVICE_API_KEY"))

# Vehicles start & end position
vehicle_start_end = [-0.4058616334605651, 39.47524916029938]

# Add marker to show end & start location
folium.Marker(
    location=list(reversed(vehicle_start_end)),
    icon=folium.Icon(color="green", icon="truck", prefix="fa"),
    draggable=True
).add_to(map)


# Define 2 vehicles. Check optimization parameters at https://github.com/VROOM-Project/vroom/blob/master/docs/API.md
vehicles = [
    ors.optimization.Vehicle(
        id=0,
        profile="driving-car",
        start=vehicle_start_end,
        end=vehicle_start_end,
        capacity=[10],
    ),
    ors.optimization.Vehicle(
        id=1,
        profile="driving-car",
        start=vehicle_start_end,
        end=vehicle_start_end,
        capacity=[10],
    ),
]

# Define job
jobs = [
    ors.optimization.Job(id=index, location=coords, amount=[1])
    for index, coords in enumerate(coords)
]

# Get optimization
optimized = client.optimization(jobs=jobs, vehicles=vehicles, geometry=True)

# Print solution on map
line_colors = ["green", "orange", "blue", "yellow"]

for route in optimized["routes"]:
    folium.PolyLine(
        locations=[
            list(reversed(coords))
            for coords in ors.convert.decode_polyline(route["geometry"])["coordinates"]
        ],
        color=line_colors[route["vehicle"]],
    ).add_to(map)

map

## Store solution

### Create entities

In [ ]:
# Based on https://ask.openrouteservice.org/t/use-optimitazion-api-with-leaflet/1550/2
import polyline
from geojson import LineString

# For each route
for route in optimized["routes"]:
    # Decode each geometry to geoJSON. Check https://pypi.org/project/polyline/
    decoded = polyline.decode(route["geometry"], 5, geojson=True)

    # Create geoJSON LineString. Check https://pypi.org/project/geojson/#linestring
    linestring = LineString(decoded)
    print(linestring)

print(optimized)